In [1]:
# so notebook can access the folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch torchaudio
!pip install datasets
!pip install pandas
!pip install transformers
!pip install evaluate
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 10.5 MB/s eta 0:00:00


In [3]:
!tar -xzvf /content/drive/MyDrive/DS565_Project/data/audio.tar.gz

Streaming output truncated to the last 5000 lines.
audio/train/Ses03M_impro02_F027.wav
audio/train/Ses03M_script03_1_F009.wav
audio/train/Ses01M_impro07_F009.wav
audio/train/Ses03M_impro02_F030.wav
audio/train/Ses04M_script03_2_M038.wav
audio/train/Ses02F_script03_2_F004.wav
audio/train/Ses04M_impro08_M006.wav
audio/train/Ses04F_impro02_M018.wav
audio/train/Ses03F_script02_1_F012.wav
audio/train/Ses04F_script02_2_F018.wav
audio/train/Ses02M_impro05_F018.wav
audio/train/Ses04F_impro05_F014.wav
audio/train/Ses03M_impro05b_M020.wav
audio/train/Ses02M_script01_1_M033.wav
audio/train/Ses05M_script02_1_F001.wav
audio/train/Ses01F_impro06_F002.wav
audio/train/Ses05M_impro04_M014.wav
audio/train/Ses02F_script01_1_F041.wav
audio/train/Ses05F_script02_2_M005.wav
audio/train/Ses04F_script01_1_M015.wav
audio/train/Ses02F_script02_1_F020.wav
audio/train/Ses02M_impro02_M018.wav
audio/train/Ses01F_script03_1_M017.wav
audio/train/Ses01M_impro06_F018.wav
audio/train/Ses01M_impro03_M002.wav
audio/train/

# Load dataset as HuggingFace Dataset

In [4]:
import os
import json
import pandas as pd
data_folder = 'audio'

with open(os.path.join(data_folder, 'metadata.json'), 'r') as f_in:
  metadata = json.load(f_in)

with open(os.path.join(data_folder, 'label_mapping.json'), 'r') as f_in:
  label_mapping = json.load(f_in)

train = pd.read_csv(os.path.join(data_folder, 'train.csv'))
val = pd.read_csv(os.path.join(data_folder, 'val.csv'))
test = pd.read_csv(os.path.join(data_folder, 'test.csv'))

print(
    len(metadata),
    len(train),
    len(val),
    len(test)
)

assert len(metadata) == len(train) + len(val) + len(test)
train.head()

10039 6023 2008 2008


,name,emotion
0,Ses02M_impro08_M013,6
1,Ses01M_impro02_M012,8
2,Ses04F_script03_2_M020,10
3,Ses04F_script01_1_F042,0
4,Ses05M_script03_2_M016,4


Save a metadata.csv file in the train,valid,test split to assign labels to file paths

In [5]:
import os
import json
import pandas as pd
data_folder = 'audio'

train = pd.read_csv(os.path.join(data_folder, 'train.csv')).rename(columns = {'emotion':'label'})
val = pd.read_csv(os.path.join(data_folder, 'val.csv')).rename(columns = {'emotion':'label'})
test = pd.read_csv(os.path.join(data_folder, 'test.csv')).rename(columns = {'emotion':'label'})

train['file_name'] = train['name'] + '.wav'
val['file_name'] = val['name'] + '.wav'
test['file_name'] = test['name'] + '.wav'

train[['file_name', 'label']].to_csv('audio/train/metadata.csv', index=False)
val[['file_name', 'label']].to_csv('audio/valid/metadata.csv',index=False)
test[['file_name', 'label']].to_csv('audio/test/metadata.csv',index=False)

In [6]:
# shape: (timestamp, label, session, improv)
metadata['Ses04F_script03_2_M020']

['[100.8400 - 107.2200]', 'xxx', 'Session4', 'Ses04F_script03_2']

Load it as a huggingface dataset

In [7]:
from datasets import load_dataset

dataset = load_dataset("audiofolder", data_dir='audio')
dataset

Resolving data files:   0%|          | 0/6024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2009 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2009 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'label'],
        num_rows: 6023
    })
    validation: Dataset({
        features: ['audio', 'label'],
        num_rows: 2008
    })
    test: Dataset({
        features: ['audio', 'label'],
        num_rows: 2008
    })
})

# Wav2Vec2-BERT

> https://huggingface.co/docs/transformers/en/model_doc/wav2vec2-bert#transformers.Wav2Vec2BertModel

In [39]:
from transformers import AutoFeatureExtractor, Wav2Vec2BertForSequenceClassification
import torch

pretrained_model = "facebook/w2v-bert-2.0"

with open(os.path.join(data_folder, 'label_mapping.json'), 'r') as f_in:
  id2label = json.load(f_in)
label2id = {label: id for id,label in id2label.items()}

num_labels = len(id2label)

device = "cuda" if torch.cuda.is_available() else "cpu"

model = Wav2Vec2BertForSequenceClassification.from_pretrained(pretrained_model,
                                                              num_labels=num_labels,
                                                              id2label=id2label,
                                                              label2id=label2id).to(device)

processor = AutoFeatureExtractor.from_pretrained(pretrained_model)

Some weights of Wav2Vec2BertForSequenceClassification were not initialized from the model checkpoint at facebook/w2v-bert-2.0 and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
processor

SeamlessM4TFeatureExtractor {
  "feature_extractor_type": "SeamlessM4TFeatureExtractor",
  "feature_size": 80,
  "num_mel_bins": 80,
  "padding_side": "right",
  "padding_value": 1,
  "processor_class": "Wav2Vec2BertProcessor",
  "return_attention_mask": true,
  "sampling_rate": 16000,
  "stride": 2
}

In [46]:
def preprocess_audio(batch):
  audio_arr = [x['array'] for x in batch['audio']]
  inputs = processor(audio_arr,
                     sampling_rate = processor.sampling_rate,
                     max_length=16000, truncation=True,
                     return_tensors='pt').to(device)
  return inputs

processed_audio = dataset.map(preprocess_audio, remove_columns = 'audio', batched=True)

Map:   0%|          | 0/6023 [00:00<?, ? examples/s]

Map:   0%|          | 0/2008 [00:00<?, ? examples/s]

Map:   0%|          | 0/2008 [00:00<?, ? examples/s]

wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


In [31]:
import evaluate
import numpy as np

accuracy = evaluate.load('accuracy')
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1_score = evaluate.load("f1")

def compute_metrics(eval_pred):
  pred = np.argmax(eval_pred.predictions, axis=1)
  metrics = {
      "accuracy": accuracy.compute(predictions=pred, references=eval_pred.label_ids),
      "precision": precision.compute(predictions=pred, references=eval_pred.label_ids),
      "recall": recall.compute(predictions=pred, references=eval_pred.label_ids),
      "f1": f1.compute(predictions=pred, references=eval_pred.label_ids)
  }
  return metrics

In [47]:
import os
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

os.environ["WANDB_PROJECT"]="Wav2Vec2-Bert"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

#TODO- Fix this error: ValueError: expected sequence of length 991 at dim 1 (got 1156)
training_args = TrainingArguments(
    report_to="wandb",
    output_dir="Wav2Vec2_1",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_audio['train'],
    eval_dataset=processed_audio['validation'],
    compute_metrics=compute_metrics
)

trainer.train()

ValueError: expected sequence of length 991 at dim 1 (got 1156)

In [54]:
len(processed_audio['train'][0]['input_features'])

1456

# Test on Validation